In [1]:
import os
import torch
import albumentations as A
import numpy as np
from albumentations.pytorch import ToTensorV2
import pandas as pd
from torch.utils.data import DataLoader

from Load_Data import ImageDataset, random_seed, get_transforms
from Preprocess import Augmentation
from Train import Trainer
from Inference import Model_Ensemble, tta, run_inference

In [2]:
train_csv_path = "../data/train_update.csv"
test_csv_path = "../data/sample_submission.csv"

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

random_seed(42) # 시드 고정

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 480  # EfficientNetV2 L 권장 입력 크기
LR = 1e-5
EPOCHS = 80
BATCH_SIZE = 10
Drop_out = 0.45
weight_decay = 1e-3
num_workers = 4
num_classes = 17
n_splits = 3  # K-fold 
patience = 5  # Early stopp

model_name = "tf_efficientnetv2_l_in21ft1k"  
run_name_prefix = "V14"
train_path = "../data/train/"
test_path = "../data/test/"
fold_paths = "../model/V14/" # 학습된 fold 모델 경로
model_save_path = "../model/V14" # 모델 저장 경로
augmented_save_path = "../data/augment_image/V14-augmented" # 증강 이미지 저장 경로
augmented_save_csv_path = "../data/augment_csv/V14-augmented.csv" # 증강 csv 저장 경로
submission_save_path = "../data/submission/V14-submission.csv" # 추론 결과

In [ ]:
# Augmentation
augmenter = Augmentation(
    train_df=train_df,
    train_path=train_path, 
    save_path=augmented_save_path,
    img_size=img_size,
    save_csv_path=augmented_save_csv_path 
)
aug_df = augmenter.run(target_count=300) # Class 별 300

# Train
trainer = Trainer(
    df=aug_df,                            # 증강된 데이터 사용
    data_path=augmented_save_path,        # 증강된 이미지 경로 사용
    model_name=model_name,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    lr=LR,
    drop_out=Drop_out,
    weight_decay=weight_decay,
    img_size=img_size,
    num_workers=num_workers,
    device=device,
    save_dir=model_save_path,
    run_name_prefix=run_name_prefix,
    num_classes=num_classes,
    n_splits=n_splits,
    patience=patience,
    k_fold=True
)

f1_df = trainer.run()
fold_weights = f1_df["f1"].values / f1_df["f1"].sum()

[INFO] Class 0: 현재 100개 → 목표 300개로 증강 중...


[Class 0] 원본 복사: 100%|██████████| 100/100 [00:00<00:00, 280.22it/s]


[INFO] Class 0: 추가 증강 200개 생성 중...


[Class 0] 증강 생성: 100%|██████████| 200/200 [00:07<00:00, 26.29it/s]


[INFO] Class 1: 현재 46개 → 목표 300개로 증강 중...


...

[Fold 1][Epoch 17/80] Validation: 100%|██████████| 170/170 [00:18<00:00,  9.25it/s]


[Fold 1] Ep17 - Train: 0.1153 | Val: 0.6652, Acc: 0.9035, F1: 0.9036


[Fold 1][Epoch 18/80] Validation: 100%|██████████| 170/170 [00:18<00:00,  9.24it/s]


[Fold 1] Ep18 - Train: 0.1114 | Val: 0.6941, Acc: 0.9018, F1: 0.9016


[Fold 1][Epoch 19/80] Training:   0%|          | 1/340 [00:00<03:57,  1.43it/s, loss=0.0482]

In [ ]:
# Ensemble
ensembler = Model_Ensemble(
    model_name=model_name,
    fold_paths_dir=fold_paths,  
    device=device,
    num_classes=num_classes,
    drop_out=Drop_out,
    fold_weights=fold_weights,
    k_fold=True  # k_fold=True / Hold_out=False
)

# Inference 
run_inference(
    ensembler=ensembler,
    submission_df=test_df.copy(),
    test_path=test_path,    
    img_size=img_size,
    save_path=submission_save_path,
    batch_size=BATCH_SIZE,                  
    num_workers=num_workers,
    tta_transforms=False # TTA 
)

Inference: 100%|██████████| 524/524 [02:30<00:00,  3.48it/s]

[✓] Saved submission to: ../data/submission/V13-submission.csv
